In [0]:
# Define Azure Storage details
storage_account_name = "etlproject02storage"
container_name = "raw"
mount_point = f"/mnt/{container_name}"
storage_account_key = ""   

# First try to unmount if it exists
try:
    dbutils.fs.unmount(mount_point)
    print(f"Unmounted existing mount: {mount_point}")
except:
    print(f"No existing mount found at: {mount_point}")


# Mount the container
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = mount_point,
    extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)

# Check mount
display(dbutils.fs.ls(mount_point))

/mnt/raw has been unmounted.
Unmounted existing mount: /mnt/raw


path,name,size,modificationTime
dbfs:/mnt/raw/data.csv,data.csv,45580638,1756188471000


In [0]:
file_path = "/mnt/raw/data.csv"

raw_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

# Create Delta Table
raw_df.write.mode("overwrite").format("delta").saveAsTable("online_retail.sales_raw")

# Display schema and sample data
print("Raw Layer Schema:")
raw_df.printSchema()
print("Sample of Raw Data:")
raw_df.show(5)


Raw Layer Schema:
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

Sample of Raw Data:
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   53

In [0]:
# Correct way
spark.table("online_retail.sales_raw").count()
display(spark.table("online_retail.sales_raw").limit(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
